In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import xgboost 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_predict
import xgboost as xgb
from sklearn.model_selection import cross_val_score
import optuna
import math
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Length	Feature	Continuous	Longest shell measurement	mm	no
# Diameter	Feature	Continuous	perpendicular to length	mm	no
# Height	Feature	Continuous	with meat in shell	mm	no
# Whole_weight	Feature	Continuous	whole abalone	grams	no
# Shucked_weight	Feature	Continuous	weight of meat	grams	no
# Viscera_weight	Feature	Continuous	gut weight (after bleeding)	grams	no
# Shell_weight	Feature	Continuous	after being dried	grams	no
# Rings	Target	Integer	+1.5 gives the age in years		no



# Decide between local or kaggle cloud storage     
if 'kaggle' in os.listdir('/'):
    print('Kaggle environment')
    data_path = '/kaggle/input/playground-series-s4e4'
else:
    data_path = 'kaggle/input/playground-series-s4e4'
    
for dirname, _, filenames in os.walk(data_path):
    for filename in filenames:
        print(os.path.join(dirname, filename)) 

kaggle/input/playground-series-s4e4/sample_submission.csv
kaggle/input/playground-series-s4e4/test.csv
kaggle/input/playground-series-s4e4/train.csv


In [6]:
# load the data
df_train = pd.read_csv(data_path + '/train.csv')
df_test = pd.read_csv(data_path + '/test.csv')
df_submission = pd.read_csv(data_path + '/sample_submission.csv')

In [7]:
pd.set_option('display.max_columns', None)
df_train.head(10)

,id,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
0,0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9
5,5,F,0.610,0.480,0.170,1.2010,0.5335,0.3135,0.3085,10
6,6,M,0.415,0.325,0.110,0.3315,0.1655,0.0715,0.1300,9
7,7,F,0.610,0.490,0.150,1.1165,0.4955,0.2945,0.2950,9
8,8,I,0.205,0.150,0.040,0.0460,0.0145,0.0105,0.0100,4
9,9,I,0.565,0.425,0.125,0.6510,0.3795,0.1420,0.1800,8


In [8]:
pd.set_option('display.max_columns', None)
df_test.head(10)

,id,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight
0,90615,M,0.645,0.475,0.155,1.2380,0.6185,0.3125,0.3005
1,90616,M,0.580,0.460,0.160,0.9830,0.4785,0.2195,0.2750
2,90617,M,0.560,0.420,0.140,0.8395,0.3525,0.1845,0.2405
3,90618,M,0.570,0.490,0.145,0.8740,0.3525,0.1865,0.2350
4,90619,I,0.415,0.325,0.110,0.3580,0.1575,0.0670,0.1050
5,90620,M,0.560,0.425,0.140,0.8105,0.3525,0.1915,0.2150
6,90621,M,0.635,0.490,0.170,1.1835,0.4605,0.2445,0.3550
7,90622,I,0.340,0.250,0.075,0.1675,0.0750,0.0330,0.0480
8,90623,I,0.485,0.370,0.110,0.5360,0.2565,0.0980,0.1490
9,90624,F,0.640,0.500,0.195,1.3380,0.6470,0.3175,0.3965


df_train.describe()

In [9]:
df_train.describe()

,id,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
count,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000
mean,45307.000000,0.517098,0.401679,0.135464,0.789035,0.340778,0.169422,0.225898,9.696794
std,26158.441658,0.118217,0.098026,0.038008,0.457671,0.204428,0.100909,0.130203,3.176221
min,0.000000,0.075000,0.055000,0.000000,0.002000,0.001000,0.000500,0.001500,1.000000
25%,22653.500000,0.445000,0.345000,0.110000,0.419000,0.177500,0.086500,0.120000,8.000000
50%,45307.000000,0.545000,0.425000,0.140000,0.799500,0.330000,0.166000,0.225000,9.000000
75%,67960.500000,0.600000,0.470000,0.160000,1.067500,0.463000,0.232500,0.305000,11.000000
max,90614.000000,0.815000,0.650000,1.130000,2.825500,1.488000,0.760000,1.005000,29.000000


In [10]:
df_test.describe()

,id,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight
count,60411.000000,60411.000000,60411.000000,60411.000000,60411.000000,60411.000000,60411.000000,60411.000000
mean,120820.000000,0.517428,0.401961,0.135751,0.790062,0.341227,0.169419,0.226125
std,17439.297893,0.117609,0.097470,0.038175,0.457591,0.204221,0.100720,0.129826
min,90615.000000,0.075000,0.055000,0.000000,0.002000,0.001000,0.000500,0.001500
25%,105717.500000,0.450000,0.345000,0.110000,0.419500,0.178500,0.086500,0.120000
50%,120820.000000,0.545000,0.425000,0.140000,0.801500,0.329000,0.166000,0.225000
75%,135922.500000,0.600000,0.470000,0.160000,1.070000,0.464500,0.232500,0.305000
max,151025.000000,0.800000,0.650000,1.095000,2.825500,1.488000,0.641500,1.004000


In [11]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60411 entries, 0 to 60410
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              60411 non-null  int64  
 1   Sex             60411 non-null  object 
 2   Length          60411 non-null  float64
 3   Diameter        60411 non-null  float64
 4   Height          60411 non-null  float64
 5   Whole weight    60411 non-null  float64
 6   Whole weight.1  60411 non-null  float64
 7   Whole weight.2  60411 non-null  float64
 8   Shell weight    60411 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 4.1+ MB


In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90615 entries, 0 to 90614
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              90615 non-null  int64  
 1   Sex             90615 non-null  object 
 2   Length          90615 non-null  float64
 3   Diameter        90615 non-null  float64
 4   Height          90615 non-null  float64
 5   Whole weight    90615 non-null  float64
 6   Whole weight.1  90615 non-null  float64
 7   Whole weight.2  90615 non-null  float64
 8   Shell weight    90615 non-null  float64
 9   Rings           90615 non-null  int64  
dtypes: float64(7), int64(2), object(1)
memory usage: 6.9+ MB


# Model Selection

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import mean_squared_log_error


# Datenvorbereitung
# Label-Encoding für die Spalte "Sex"
le = LabelEncoder()
df_train['Sex'] = le.fit_transform(df_train['Sex'])
df_test['Sex'] = le.transform(df_test['Sex'])

# Features und Ziel definieren
X = df_train.drop(['id', 'Rings'], axis=1)  # "id" und "Rings" werden entfernt
y = df_train['Rings']

# Für den Test-Datensatz:
X_test = df_test.drop(['id'], axis=1)

# Datenaufteilung in Training und Validierung
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost Modell erstellen
model = xgb.XGBRegressor(
    objective='reg:squarederror',  # RMSLE verwendet den squared error als Basis
    n_estimators=1000,  # Anzahl der Bäume
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# Modell trainieren
model.fit(X_train, y_train, 
          eval_set=[(X_val, y_val)],
          early_stopping_rounds=10,
          verbose=True)

# Vorhersagen und Bewertung mit RMSLE
y_val_pred = model.predict(X_val)
rmsle = np.sqrt(mean_squared_log_error(y_val, y_val_pred))
print(f'RMSLE auf dem Validierungsset: {rmsle}')

# Vorhersagen auf den Testdaten
y_test_pred = model.predict(X_test)

# Ergebnisse speichern
output = pd.DataFrame({'id': df_test['id'], 'Rings': y_test_pred})
output.to_csv('kaggle/input/submission.csv', index=False)


[0]	validation_0-rmse:3.04562
[1]	validation_0-rmse:2.87862
[2]	validation_0-rmse:2.74367
[3]	validation_0-rmse:2.61711
[4]	validation_0-rmse:2.50994
[5]	validation_0-rmse:2.42490
[6]	validation_0-rmse:2.35425
[7]	validation_0-rmse:2.28431
[8]	validation_0-rmse:2.23324
[9]	validation_0-rmse:2.18290
[10]	validation_0-rmse:2.14085
[11]	validation_0-rmse:2.10568


/home/dan/anaconda3/lib/python3.11/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[12]	validation_0-rmse:2.07648
[13]	validation_0-rmse:2.04957
[14]	validation_0-rmse:2.02904
[15]	validation_0-rmse:2.00899
[16]	validation_0-rmse:1.99285
[17]	validation_0-rmse:1.97944
[18]	validation_0-rmse:1.97099
[19]	validation_0-rmse:1.96030
[20]	validation_0-rmse:1.95062
[21]	validation_0-rmse:1.94403
[22]	validation_0-rmse:1.93776
[23]	validation_0-rmse:1.93216
[24]	validation_0-rmse:1.92776
[25]	validation_0-rmse:1.92346
[26]	validation_0-rmse:1.91960
[27]	validation_0-rmse:1.91724
[28]	validation_0-rmse:1.91501
[29]	validation_0-rmse:1.91224
[30]	validation_0-rmse:1.91009
[31]	validation_0-rmse:1.90730
[32]	validation_0-rmse:1.90507
[33]	validation_0-rmse:1.90390
[34]	validation_0-rmse:1.90224
[35]	validation_0-rmse:1.90025
[36]	validation_0-rmse:1.89945
[37]	validation_0-rmse:1.89782
[38]	validation_0-rmse:1.89703
[39]	validation_0-rmse:1.89672
[40]	validation_0-rmse:1.89526
[41]	validation_0-rmse:1.89362
[42]	validation_0-rmse:1.89247
[43]	validation_0-rmse:1.89139
[44]	val